<a href="https://colab.research.google.com/github/LamaAldakhil/ScCvT/blob/main/ScCvT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from sklearn.metrics import f1_score, roc_auc_score
from torch import nn

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
# Login to huggingface account
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
%%capture
!sudo apt -qq install git-lfs
!git config --global credential.helper store

In [ ]:
# Define the number of folds
num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

In [ ]:
# Store each fold metric in a list to calculate the mean metrics
fold_metrics = []

In [ ]:
import numpy as np
from datasets import load_dataset
from sklearn.utils.class_weight import compute_class_weight

# Load the dataset
dataset = load_dataset("imagefolder", data_dir="/content/drive/MyDrive/HAM/train")

# Get the 'label' feature of the dataset
label_feature = dataset['train'].features['label']

# Get the class labels
class_labels = label_feature.names

# Get the class indices
class_indices = label_feature.str2int(class_labels)

# Compute the class weights
class_weights = compute_class_weight('balanced', classes=class_indices, y=dataset['train']['label'])

# Normalize the class weights
class_weights_normalized = class_weights / np.sum(class_weights)

# Create a dictionary mapping class labels to their corresponding normalized weights
class_weights_dict_normalized = dict(zip(class_labels, class_weights_normalized))

# Print the normalized class weights
print(class_weights_dict_normalized)

Resolving data files:   0%|          | 0/9581 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

{'akiec': 0.13174947960827396, 'bcc': 0.08356095399279632, 'bkl': 0.0390812833050931, 'df': 0.3734811334982375, 'mel': 0.06325527297834656, 'nv': 0.006404761460229245, 'vasc': 0.30246711515702335}


In [ ]:
import torch
# Convert the normalized class weights to a PyTorch tensor
class_weights_tensor = torch.tensor(class_weights_normalized, dtype=torch.float32)

# Print the normalized class weights tensor
print(class_weights_tensor)

tensor([0.1317, 0.0836, 0.0391, 0.3735, 0.0633, 0.0064, 0.3025])


In [ ]:
labels = dataset["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

id2label[0]

'akiec'

In [ ]:
print(class_weights)

[ 4.19851008  2.66286826  1.24541791 11.90186335  2.01577951  0.20410294
  9.638833  ]


In [ ]:
# Load model to fine-tune
import transformers
from transformers import AutoImageProcessor

model_checkpoint = "microsoft/cvt-13"
image_processor = AutoImageProcessor.from_pretrained(model_checkpoint)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [ ]:
import torch

def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")
f1=evaluate.load('f1')

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, confusion_matrix
from typing import Dict
import numpy as np
import torch

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Convert logits to probabilities using softmax
    probabilities = torch.softmax(torch.tensor(logits), dim=-1).numpy()

    # Compute F1 score
    f1 = f1_score(y_true=labels, y_pred=predictions, average='weighted')

    # Compute ROC-AUC score using One-vs-One approach
    roc_auc = roc_auc_score(y_true=labels, y_score=probabilities, multi_class='ovo', average='weighted')

    # Compute accuracy
    accuracy = accuracy_score(y_true=labels, y_pred=predictions)


    return {"f1": f1, "roc_auc": roc_auc, "accuracy": accuracy}


In [ ]:
import torch
from torch import nn
from transformers import Trainer

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # move weights_tensor to the model's device
        weights_tensor_device = class_weights_tensor.to(model.device)
        # compute custom loss (suppose one has 3 labels with different weights)
        loss_fct = nn.CrossEntropyLoss(weight=weights_tensor_device)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

In [ ]:
import json
import numpy as np

class NumpyArrayEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NumpyArrayEncoder, self).default(obj)

In [ ]:
# Preprocessing
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomResizedCrop,
    Resize,
    ToTensor,
)

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
if "height" in image_processor.size:
    size = (image_processor.size["height"], image_processor.size["width"])
    crop_size = size
    max_size = None
elif "shortest_edge" in image_processor.size:
    size = image_processor.size["shortest_edge"]
    crop_size = (size, size)
    max_size = image_processor.size.get("longest_edge")

train_transforms = Compose(
        [
            RandomResizedCrop(crop_size),
            RandomHorizontalFlip(),
            ToTensor(),
            normalize,
        ]
    )

val_transforms = Compose(
        [
            Resize(size),
            CenterCrop(crop_size),
            ToTensor(),
            normalize,
        ]
    )

def preprocess_train(example_batch):
    """Apply train_transforms across a batch."""
    example_batch["pixel_values"] = [
        train_transforms(image.convert("RGB")) for image in example_batch["image"]
    ]
    example_batch["labels"] = example_batch["label"]
    return example_batch

def preprocess_val(example_batch):
    """Apply val_transforms across a batch."""
    example_batch["pixel_values"] = [val_transforms(image.convert("RGB")) for image in example_batch["image"]]
    example_batch["labels"] = example_batch["label"]
    return example_batch

In [ ]:
# Define the training arguments for the model
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="ScCvT_K-fold",
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    weight_decay=0.01,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=32,
    num_train_epochs=65,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    push_to_hub=True,
)


In [ ]:
# Change starting fold in case the session disconnects during training
for fold, (train_indices, val_indices) in enumerate(kf.split(dataset['train']), start=4):
    print(f"Processing fold {fold + 1}/{num_folds}")

    # Stop training after completing the 5th fold
    if fold + 1 >= 6:
        break

    # Create train and validation datasets for the current fold
    train_ds_fold = dataset['train'].select(train_indices)
    val_ds_fold = dataset['train'].select(val_indices)

    # Set the transforms for the current fold's datasets
    train_ds_fold.set_transform(preprocess_train)
    val_ds_fold.set_transform(preprocess_val)

    # Instantiate a new model for the current fold
    model = AutoModelForImageClassification.from_pretrained(
        model_checkpoint,
        label2id=label2id,
        id2label=id2label,
        ignore_mismatched_sizes=True,
    )

    # Create a trainer with the current fold's datasets
    trainer = CustomTrainer(
        model=model,
        args=training_args,
        data_collator=collate_fn,
        train_dataset=train_ds_fold,
        eval_dataset=val_ds_fold,
        tokenizer=image_processor,
        compute_metrics=compute_metrics,
    )

    # Train the model for the current fold
    train_results = trainer.train()

    # Save and log the metrics for the current fold
    trainer.save_model()
    trainer.log_metrics("train", train_results.metrics)
    trainer.save_metrics("train", train_results.metrics)
    trainer.save_state()

    # Evaluate the model for the current fold and append the metrics to the fold_metrics list
    metrics = trainer.evaluate()
    fold_metrics.append(metrics)

    # Some nice-to-haves for the current fold
    trainer.log_metrics("eval", metrics)
    trainer.save_metrics("eval", metrics)

    # Push the current fold's model to the Hugging Face Hub
    trainer.push_to_hub(model_name=f"ScCvT_fold_{fold + 1}")

Processing fold 5/5


Some weights of CvtForImageClassification were not initialized from the model checkpoint at microsoft/cvt-13 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 384]) in the checkpoint and torch.Size([7, 384]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([7]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/content/ScCvT_K-fold is already a clone of https://huggingface.co/LamaAldakhil/ScCvT_K-fold. Make sure you pull the latest changes with `repo.git_pull()`.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,1.885800,1.833615,0.375627,0.676968,0.306208
2,1.757300,1.574776,0.590367,0.801279,0.534168
3,1.604200,1.244802,0.664769,0.878221,0.627021
4,1.401100,1.072772,0.685598,0.916198,0.645801
5,1.358800,0.868034,0.743245,0.932916,0.713093
6,1.229100,0.843072,0.737759,0.939987,0.705269
7,1.145000,0.850067,0.735482,0.945275,0.697444
8,1.065200,0.747060,0.755868,0.953288,0.725091
9,1.017400,0.559204,0.814025,0.963390,0.800209
10,0.892000,0.678514,0.772567,0.961355,0.748044


Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 1.00/75.1M [00:00<?, ?B/s]

...

***** train metrics *****
  epoch                    =         65.0
  total_flos               = 8219203090GF
  train_loss               =       0.7256
  train_runtime            =   4:48:22.58
  train_samples_per_second =       28.791
  train_steps_per_second   =        0.225


***** eval metrics *****
  epoch                   =       65.0
  eval_accuracy           =     0.9103
  eval_f1                 =       0.91
  eval_loss               =     0.3026
  eval_roc_auc            =      0.985
  eval_runtime            = 0:00:38.12
  eval_samples_per_second =     50.276
  eval_steps_per_second   =      1.574


Upload file runs/Jun02_19-15-42_82fc93d52d9f/events.out.tfevents.1685750720.82fc93d52d9f.173035.2: 100%|######…

To https://huggingface.co/LamaAldakhil/ScCvT_K-fold
   f6019e0..6f328b6  main -> main

   f6019e0..6f328b6  main -> main

To https://huggingface.co/LamaAldakhil/ScCvT_K-fold
   6f328b6..feafe5b  main -> main

   6f328b6..feafe5b  main -> main



Processing fold 6/5


In [ ]:
# Calculate the mean metrics across all folds
mean_metrics = {}
for metric in fold_metrics[0].keys():
    mean_metrics[metric] = np.mean([fold[metric] for fold in fold_metrics])
print("Mean metrics:", mean_metrics)

In [ ]:
# Calculate mean metrics in case the session disconnects during the training of the folds
import numpy as np

# Manually specify the metric values for each fold
accuracy_values = [0.904, 0.9113, 0.9071, 0.9124, 0.9103]
f1_values = [0.9032, 0.9119, 0.9053, 0.9121, 0.91]
roc_auc_values = [0.9831, 0.985, 0.9847, 0.9857, 0.985]
eval_loss_values = [0.3209, 0.2966, 0.2996, 0.2933, 0.3026]
train_loss_values = [0.7327, 0.738, 0.7357, 0.7418, 0.7256]

# Calculate the mean values for each metric
mean_accuracy = np.mean(accuracy_values)
mean_f1 = np.mean(f1_values)
mean_roc_auc = np.mean(roc_auc_values)
mean_eval_loss = np.mean(eval_loss_values)
mean_train_loss = np.mean(train_loss_values)

# Print the mean metric values
print(f'Mean accuracy: {mean_accuracy:.4f}')
print(f'Mean F1 score: {mean_f1:.4f}')
print(f'Mean ROC-AUC: {mean_roc_auc:.4f}')
print(f'Mean evaluation loss: {mean_eval_loss:.4f}')
print(f'Mean training loss: {mean_train_loss:.4f}')

Mean accuracy: 0.9090
Mean F1 score: 0.9085
Mean ROC-AUC: 0.9847
Mean evaluation loss: 0.3026
Mean training loss: 0.7348
